# 데이터 불러오기 및 병합

In [ ]:
import pandas as pd

df1 = pd.read_excel('../최종 파일/84data_final.xlsx', index_col=0)
df1.head()

In [ ]:
df2 = pd.read_excel('../최종 파일/85data_final.xlsx', index_col=0)
df2.head()

In [ ]:
product_85 = df2['product']
df_85 = pd.DataFrame({'hs code':df2['hs code'], 'description':df2['description'], 'reason':df2['reason'], 'name_des':df2['name_des']})
df_85.insert(1, 'product', product_85)
df_85.head()

In [ ]:
df = pd.concat([df1, df_85])
df.head()

In [ ]:
len(df['hs code'].unique())

# 데이터 전처리

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')  # pos_tag를 위한 데이터 다운로드

# Initialize WordNet Lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')
stop_words.append('x')
stop_words.append('w')

lemmatized_des = []
for i in df['name_des']:
    # 텍스트 전처리: 소문자 변환, 토큰화, 불용어 제거
    tokens = word_tokenize(i.lower())
    tagged_tokens = nltk.pos_tag(tokens)  # 단어 토큰에 품사 태깅 추가

    # 품사를 고려하여 단어의 원형 복원 (명사와 동사만 고려)
    filtered_tokens = []
    for word, tag in tagged_tokens:
        if (tag.startswith('NN') or tag.startswith('VB')) and word not in stop_words:  # 명사 또는 동사인 경우
            lemma = lemmatizer.lemmatize(word, pos='n' if tag.startswith('NN') else 'v')
            filtered_tokens.append(lemma)
        elif word.isalpha() and word not in stop_words:
            filtered_tokens.append(word)
    lemmatized_des.append(' '.join(filtered_tokens))

print(lemmatized_des[:4])

In [ ]:
preprocessed_df = pd.DataFrame({'hs code': df['hs code'], 'name_des':lemmatized_des})
preprocessed_df

# 종속변수, 독립변수 나누기

In [ ]:
x_train = preprocessed_df['name_des']
y_train = preprocessed_df['hs code']

print(len(x_train))
print(len(y_train))

# 벡터라이저

In [ ]:
tfidf_vec = TfidfVectorizer()
tfidf_vec.fit(x_train)
x_train_tfidf_vec = tfidf_vec.transform(x_train) 

# 모델 학습 - LSVC

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score,classification_report

svc = LinearSVC(C=1)
svc.fit(x_train_tfidf_vec, y_train)

# 모델 학습 - 랜덤 포레스트
- 파라메터값은 정확히 모르겠어서 수정 부탁

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rmc =RandomForestClassifier(n_estimators=1000, max_depth=500)
rmc.fit(x_train_tfidf_vec,y_train)

# 모델 학습 - 로지스틱회귀분류
- 파라메터값은 정확히 모르겠어서 수정 부탁

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='lbfgs', multi_class='ovr')
lr.fit(x_train_tfidf_vec,y_train)

In [ ]:
# 피클로 모델 저장
import pickle

with open('./drive/MyDrive/DIMA3/lsvc.pkl', 'wb') as file:
    pickle.dump(lsvc, file)

In [ ]:
with open('./drive/MyDrive/DIMA3/lsvc.pkl', 'wb') as file:
    pickle.dump(rmc, file)

In [ ]:
with open('./drive/MyDrive/DIMA3/lsvc.pkl', 'wb') as file:
    pickle.dump(lr, file)

# 테스트 데이터 적용 코드

In [ ]:
test = pd.read_excel('../최종 파일/usa_test_data.xlsx', index_col=0)
test.head()

In [ ]:
# 테스트 인덱스 값 맞추기
test.reset_index(drop=True, inplace=True)
test

In [ ]:
# 학습에 적용된 벡터라이저로 테스트 데이터에도 적용
test_tfidf_vec = tfidf_vec.transform(test)

In [ ]:
# 전처리된 테스트 데이터를 예측 모델에 적용
y_hat_lsvc = svc.predict(test_tfidf_vec)
#y_hat_rmc = rmc.predict(test_tfidf_vec)
y_hat_lr = lr.predict(test_tfidf_vec)

In [ ]:
# LSVC 정확도 체크
cnt = 0
for i, v in zip(test['hs code'], y_hat_lsvc):
    print(i, v)
    if i == v:
        cnt += 1

print('LSVC 정확도:', cnt/len(test))

In [ ]:
# RandomForestClassifier 정확도 체크
cnt = 0
for i, v in zip(test['hs code'], y_hat_rmc):
    if i == v:
        cnt += 1

print('LSVC 정확도:', cnt/len(test))

In [ ]:
# LogisticRegression 정확도 체크
cnt = 0
for i, v in zip(test['hs code'], y_hat_lr):
    if i == v:
        cnt += 1

print('LSVC 정확도:', cnt/len(test))